In [24]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint
import data

In [25]:
vocab_size = 324
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
print len(d)
x, y = d[10:20], d[11:21]
print [np.argmax(j) for j in x]
print [np.argmax(j) for j in y]

324
735
[29, 56, 8, 1, 82, 9, 45, 22, 24, 67]
[56, 8, 1, 82, 9, 45, 22, 24, 67, 2]


In [3]:
# dirrs = ['sentiment/train/', 'sentiment/test/']
# sent = []
# for dirr in dirrs:
#     print dirr
#     l = listdir(dirr+'pos')
#     print 'pos'
#     for r in l:
#         t = codecs.open(dirr+'pos/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))
#     l = listdir(dirr+'neg')
#     print 'neg'
#     for r in l:
#         t = codecs.open(dirr+'neg/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))

In [4]:
# m = Word2Vec.load('embedding.model')
# data = []
# for s in sent:
#     a = [m[x] for x in s]
#     data.append(a)
# data = np.array(data)

In [5]:
# def sam(d):
#     r = randint(0, data.shape[0] - 1)
#     t = d[r]
#     x = t[:-1]
#     y = t[1:]
#     return np.array(x), np.array(y)

# s = sample(data)
# s[0].shape, s[1].shape

In [50]:
# hyper-parameters
epochs = 10000
hidden_layer = 128
inp_out_size = 324
learning_rate = 0.001
num_steps = 13

In [36]:
# Wxh = tf.Variable([[.1, .2], [.3, .4], [.5, .6], [.7, .8]], dtype=tf.float32)
# Whh = tf.Variable([[.1, .2], [.3, .4]], dtype=tf.float32)
# Why = tf.Variable([[.1, .2, .3, .4], [.4, .5, .6, .7]], dtype=tf.float32)
Wxh = tf.Variable(tf.random_normal([inp_out_size, hidden_layer], mean=0, stddev=0.001))
Whh = tf.Variable(tf.random_normal([hidden_layer, hidden_layer], mean=0, stddev=0.001))
Why = tf.Variable(tf.random_normal([hidden_layer, inp_out_size], mean=0, stddev=0.001))
bh = tf.Variable(tf.zeros([hidden_layer]), dtype=tf.float32)
by = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [37]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    h = tf.tanh((tf.matmul(p, Whh))  + tf.matmul(i, Wxh) + bh)
    h = tf.reshape(h, [hidden_layer])
    return h

In [41]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
initial = tf.placeholder(shape=[hidden_layer], dtype=tf.float32)
states = tf.scan(recurrence, a, initializer=initial)
outputs = tf.nn.softmax(tf.matmul(states, Why) + by)
loss = -tf.reduce_sum(b*tf.log(outputs))
# loss = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(outputs, b))))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

# clipping gradients between -1 and 1.
# trainable_variables = tf.trainable_variables()
# grad_var_pairs = optimizer.compute_gradients(loss, trainable_variables)
# print grad_var_pairs[1]
# clipped_grad_var_pairs = [(tf.clip_by_value(gv[0], -1, 1), gv[1]) for gv in grad_var_pairs]
# optimize_op = optimizer.apply_gradients(clipped_grad_var_pairs)

optimize_op = optimizer.minimize(loss)

In [42]:
def generate(sess, n):
    x, _ = data.sample(d, 1)
    gen = [id_to_token[np.argmax(x[0])]]
    h = np.zeros(hidden_layer)
    for i in range(n):
        o, h = sess.run([outputs, states], {a:x, initial: h})
        h = h.reshape(hidden_layer)
        o = np.argmax(o[0])
        gen.append(id_to_token[o])
        x = [0] * inp_out_size
        x[o] = 1
#         print np.argmax(x)
        x = [x]
    print ' '.join(gen)

In [43]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
ix = 0
smooth_loss = -np.log(1.0 / vocab_size)*num_steps

In [51]:
print epochs
h = np.zeros(hidden_layer)
for i in range(epochs):
#     x, y = data.sample(d, num_steps)
    if ix + num_steps >= len(d):
        ix = 0
        
    h = np.zeros(hidden_layer)
    x, y = d[ix : ix + num_steps], d[ix + 1 : ix + num_steps + 1]   
    l, h, _ = sess.run([loss, states, optimize_op], {a: x, b: y, initial: h}) 
    smooth_loss = smooth_loss * 0.999 + l * 0.001
    if i % 1000 == 0:
        print 'epoch {0}, loss = {1}'.format(i, smooth_loss)
#         print sess.run(Whh)[0][:30]
    ix += num_steps
        
#         generate(sess,40)

10000
epoch 0, loss = 8.43297839833
epoch 1000, loss = 7.87199647324
epoch 2000, loss = 7.38318113127
epoch 3000, loss = 6.92831863497
epoch 4000, loss = 8.80318795135
epoch 5000, loss = 8.47478664454
epoch 6000, loss = 6.78799833011
epoch 7000, loss = 5.92561129611
epoch 8000, loss = 5.41741338838
epoch 9000, loss = 5.08114284208


In [52]:
generate(sess, 500)


 She will mix these pleasures up 
 Like three fit wines in a : 
 Let would faggot earth thy these 
 Quiet on her mossy nest ; 's Fancy all fade shalt 
 lip queen , 
 From dewy ploughboy and morn feet , 
 While shalt joys the goblet pelteth . 
 Let would faggot earth rest these 
 Quiet on the mossy skin ; 
 Summer the joys as winter-thin shuffled chide , 
 All queen faggot mind these caw 
 Quiet the the mossy 
 And , joys : and by use , 
 All through faggot mind From 
 Quiet the morn or thorny swarm ; 
 All ripe heaped alway 
 Sapphire queen and these shower Quiet , her loose ; 
 While the joys as winter-thin shuffled chide 
 Let would heaped earth rest these 
 Quiet on the mossy skin ; 
 Pleasure as is all home : 
 Blushing through the mind these From , 
 antheming the morn ? 
 And , in the same moment , 
 She will God earth alarm these 
 
 Quiet the morn : 
 And every leaf , and every flower 
 
 lip queen 'll dart these caw the morn and nest , the face 
 Acorns hyacinth , alway 
 She

In [53]:
sess.run(Whh)

array([[ 0.13754342,  0.19704947, -0.05745095, ...,  0.16697039,
        -0.08112098, -0.0941937 ],
       [-0.03235122, -0.04876881,  0.05504122, ..., -0.26356348,
        -0.08242013, -0.02991492],
       [ 0.10348038, -0.13705294,  0.08280424, ...,  0.24422346,
        -0.07879512,  0.01655063],
       ..., 
       [ 0.12828396, -0.08228143,  0.23148909, ...,  0.19258361,
         0.03882705,  0.0760185 ],
       [ 0.17688558,  0.07320131,  0.29498637, ...,  0.09559939,
         0.13998733, -0.15826966],
       [-0.21004789,  0.13676059, -0.11080284, ..., -0.07399724,
        -0.10957402,  0.20072849]], dtype=float32)